# Classifier training

**Inputs:**
- data/heart_disease_cleaned.csv
- data/fair_heart_disease_full.csv
- data/counterfactual_heart_disease_full.csv

**Outputs:**
- results/perf_metrics.csv

## Setup and imports

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import userdata
  from google.colab import drive
  drive.mount('/content/drive')
  PROJECT_ROOT = userdata.get('PROJECT_ROOT')
else:
  PROJECT_ROOT = '../'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q semopy

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.colab import output
# output.enable_custom_widget_manager()
output.disable_custom_widget_manager()

sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1)

In [ ]:
heart_disease = pd.read_csv(f'{PROJECT_ROOT}/data/heart_disease_cleaned.csv')
fair_heart_disease = pd.read_csv(f'{PROJECT_ROOT}/data/fair_heart_disease_full.csv')
cf_heart_disease = pd.read_csv(f'{PROJECT_ROOT}/data/cf_heart_disease_full.csv')

### Function library

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,\
 recall_score, roc_auc_score
import semopy

def train_random_forest(X_train, y_train, X_test, y_test):
  '''
    Trains a sklearn RandomForestClassifier on the given training data,\
     optimised hyperparameters with 3-fold GridSearchCV

     Inputs:
       X_train: training features
       y_train: training labels
       X_test: test features
       y_test: test labels

     Outputs:
       rf: trained RandomForestClassifier
       y_pred: predicted labels
       y_pred_proba: predicted probabilities
  '''
  param_grid = {
    "max_depth": [5, 10, 20, None],
    "max_features": ["sqrt", "log2"],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
  }

  #create the RF classifier
  rf = RandomForestClassifier(random_state=4, n_estimators=100)

  #create the grid search
  rf_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid,
                               n_iter=10, scoring='roc_auc',
                               cv=3, n_jobs=-1, random_state=4)

  #fit the grid search
  rf_search.fit(X_train, y_train)
  y_pred = rf_search.predict(X_test)
  y_pred_proba = rf_search.predict_proba(X_test)[:,1]

  return [rf_search, y_pred, y_pred_proba]

def get_perf_metrics(y_true, y_pred, y_pred_proba):
  '''
    Calculates the performance metrics for a given set of predictions.

    Inputs
      y_true: true labels
      y_pred: predicted labels
      y_pred_proba: predicted probabilities

    Outputs
      accuracy: accuracy score
      roc_auc: ROC AUC (Receiver Operating Characteristic Area Under the Curve)
      FNR: False Negative Rate
      FPR: False Positive Rate
      tn: True Negatives
      fp: False Positives
      fn: False Negatives
      tp: True Positives
  '''
  accuracy = accuracy_score(y_true, y_pred)
  roc_auc = roc_auc_score(y_true, y_pred_proba)
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  FNR = fn / (fn + tp)
  FPR = fp / (fp + tn)

  return [accuracy, roc_auc, FNR, FPR, tn, fp, fn, tp]

def get_causal_model_params(X_train, y_train, y_pred_proba, protected_attribute):
  '''
    Trains a semopy causal model on the given training data and predictions,\
     for the following linear causal model:\
      y_pred ~ beta0 + beta1*y_true + beta2*protected_attribute,\
     to identify the causal relationship between the protected attribute\
      and the predicted outcome.

     Inputs
       X_train: training features
       y_train: training labels
       y_pred_proba: predicted probabilities
       protected_attribute: name of the protected attribute

     Outputs
       beta2: coefficient of the causal relationship between\
        the protected attribute and the predicted outcome
       beta2_pvalue: p-value of the causal relationship
  '''
  causal_features = pd.DataFrame()
  causal_features['protected_attribute'] = X_train[protected_attribute]
  causal_features['y_true'] = y_train
  causal_features['y_pred'] = y_pred_proba

  model_desc='''
    y_true ~ protected_attribute
    y_pred ~ y_true + protected_attribute
  '''

  causal_model = semopy.Model(model_desc)
  causal_model.fit(causal_features)
  causal_params = causal_model.inspect()

  # Retrieve the coefficients of the causal model
  beta2 = causal_params.loc[(causal_params.rval == "protected_attribute") &
                            (causal_params.lval == "y_pred"),'Estimate'].values[0]
  beta2_pvalue = causal_params.loc[(causal_params.rval == "protected_attribute") &
                            (causal_params.lval == "y_pred"),'p-value'].values[0]

  return [beta2, beta2_pvalue]


## Model training

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import barnard_exact

# baseline features and target class

X = heart_disease.drop(['cvd'], axis=1)
y = heart_disease['cvd']
X_cf = cf_heart_disease.drop(['cvd','U'], axis=1)
y_cf = cf_heart_disease['cvd']

# Bootstrapping approach with N_RUNS runs and a 70/30 split for training and test
N_RUNS = 50

sss = StratifiedShuffleSplit(n_splits=N_RUNS, test_size=0.3, random_state=42)

perf_metrics = []

for i, (train_index, test_index) in tqdm(enumerate(sss.split(X, y)), total=N_RUNS, desc="Running simulations"):

  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # Create the equivalent fair training and test sets
  fair_X_train = fair_heart_disease.loc[fair_heart_disease['ID'].isin(train_index),
                                        ['U','age']]
  fair_X_test = fair_heart_disease.loc[fair_heart_disease['ID'].isin(test_index),
                                       ['U','age']]
  fair_y_train = fair_heart_disease.loc[fair_heart_disease['ID'].isin(train_index), 'cvd']
  fair_y_test = fair_heart_disease.loc[fair_heart_disease['ID'].isin(test_index), 'cvd']

  # Train the baseline and fair models
  rf, y_pred, y_pred_proba = train_random_forest(X_train, y_train, X_test, y_test)
  fair_rf, fair_y_pred, fair_y_pred_proba = train_random_forest(
      fair_X_train, fair_y_train, fair_X_test, fair_y_test)

  #GLOBAL PERFORMANCE METRICS
  accuracy, roc_auc, FNR, FPR,*_ = get_perf_metrics(y_test, y_pred, y_pred_proba)
  fair_accuracy, fair_roc_auc, fair_FNR, fair_FPR,*_ = get_perf_metrics(fair_y_test, fair_y_pred, fair_y_pred_proba)

  # COUNTERFACTUAL PREDICTIONS for the baseline model
  X_cf_test = X_cf.iloc[test_index]
  y_cf_test = y_cf.iloc[test_index]

  y_cf_pred = rf.predict(X_cf_test)

  # STRATIFIED PERFORMANCE AND FAIRNESS
  # Baseline audit dataset
  baseline_audit_df = X_test.copy()
  baseline_audit_df['y_true'] = y_test
  baseline_audit_df['y_pred'] = y_pred
  baseline_audit_df['y_pred_proba'] = y_pred_proba
  baseline_audit_df['y_cf_pred'] = y_cf_pred

  baseline_male_df = baseline_audit_df[baseline_audit_df['sex'] == 1]
  baseline_female_df = baseline_audit_df[baseline_audit_df['sex'] == 0]

  # Fair audit dataset
  fair_audit_df = fair_heart_disease.loc[fair_heart_disease['ID'].isin(test_index),
                                       ['U','age','sex']].copy()
  fair_audit_df['y_true'] = fair_y_test
  fair_audit_df['y_pred'] = fair_y_pred
  fair_audit_df['y_pred_proba'] = fair_y_pred_proba
  fair_male_df = fair_audit_df[fair_audit_df['sex'] == 1]
  fair_female_df = fair_audit_df[fair_audit_df['sex'] == 0]

  ### STRATIFIED PERFORMANCE AUDIT
  # Baseline Model:
  accuracy_m, roc_auc_m, FNR_m, FPR_m, tn_m, fp_m, fn_m, tp_m = get_perf_metrics(
      baseline_male_df['y_true'],
      baseline_male_df['y_pred'],
      baseline_male_df['y_pred_proba'])

  accuracy_f, roc_auc_f, FNR_f, FPR_f, tn_f, fp_f, fn_f, tp_f = get_perf_metrics(
      baseline_female_df['y_true'],
      baseline_female_df['y_pred'],
      baseline_female_df['y_pred_proba'])

  # Fair Model before correction of direct bias:
  fair_accuracy_m, fair_roc_auc_m, fair_FNR_m, fair_FPR_m, *_ = get_perf_metrics(
      fair_male_df['y_true'],
      fair_male_df['y_pred'],
      fair_male_df['y_pred_proba'])

  fair_accuracy_f, fair_roc_auc_f, fair_FNR_f, fair_FPR_f, *_ = get_perf_metrics(
      fair_female_df['y_true'],
      fair_female_df['y_pred'],
      fair_female_df['y_pred_proba'])

  ### COUNTERFACTUAL FAIRNESS METRICS

  #### Baseline Model:

  # Frequency of male individuals with a counterfactually flipped prediction
  # from y=0 to y=1
  flipped_pos_m = (baseline_male_df['y_pred'] == 0) & (baseline_male_df['y_cf_pred'] == 1)
  flipped_pos_m_freq = baseline_male_df[flipped_pos_m].shape[0] / baseline_male_df.shape[0]

  # Frequency of male individuals with a counterfactually flipped prediction
  # from y=1 to y=0
  flipped_neg_m = (baseline_male_df['y_pred'] == 1) & (baseline_male_df['y_cf_pred'] == 0)
  flipped_neg_m_freq = baseline_male_df[flipped_neg_m].shape[0] / baseline_male_df.shape[0]

  # Frequency of male individuals with a counterfactually flipped prediction
  # from y=0 to y=1
  flipped_pos_f = (baseline_female_df['y_pred'] == 0) & (baseline_female_df['y_cf_pred'] == 1)
  flipped_pos_f_freq = baseline_female_df[flipped_pos_f].shape[0] / baseline_female_df.shape[0]

  # Frequency of male individuals with a counterfactually flipped prediction
  # from y=1 to y=0
  flipped_neg_f = (baseline_female_df['y_pred'] == 1) & (baseline_female_df['y_cf_pred'] == 0)
  flipped_neg_f_freq = baseline_female_df[flipped_neg_f].shape[0] / baseline_female_df.shape[0]



  perf_metrics.append({
      'run': i,
      'accuracy': accuracy,
      'roc_auc': roc_auc,
      'FNR': FNR,
      'FPR': FPR,
      'fair_accuracy': fair_accuracy,
      'fair_roc_auc': fair_roc_auc,
      'fair_FNR': fair_FNR,
      'fair_FPR': fair_FPR,
      'accuracy_m': accuracy_m,
      'accuracy_f': accuracy_f,
      'accuracy_diff': accuracy_m - accuracy_f,
      'roc_auc_m': roc_auc_m,
      'roc_auc_f': roc_auc_f,
      'roc_auc_diff': roc_auc_m - roc_auc_f,
      'FNR_m': FNR_m,
      'FNR_f': FNR_f,
      'FNR_diff': FNR_m - FNR_f,
      'FPR_m': FPR_m,
      'FPR_f': FPR_f,
      'FPR_diff': FPR_m - FPR_f,
      'fair_accuracy_m': fair_accuracy_m,
      'fair_accuracy_f': fair_accuracy_f,
      'fair_accuracy_diff': fair_accuracy_m - fair_accuracy_f,
      'fair_roc_auc_m': fair_roc_auc_m,
      'fair_roc_auc_f': fair_roc_auc_f,
      'fair_roc_auc_diff': fair_roc_auc_m - fair_roc_auc_f,
      'fair_FNR_m': fair_FNR_m,
      'fair_FNR_f': fair_FNR_f,
      'fair_FNR_diff': fair_FNR_m - fair_FNR_f,
      'fair_FPR_m': fair_FPR_m,
      'fair_FPR_f': fair_FPR_f,
      'fair_FPR_diff': fair_FPR_m - fair_FPR_f,
      'flipped_pos_m': flipped_pos_m_freq,
      'flipped_neg_m': flipped_neg_m_freq,
      'flipped_pos_f': flipped_pos_f_freq,
      'flipped_neg_f': flipped_neg_f_freq
  })


Running simulations: 100%|██████████| 50/50 [26:06<00:00, 31.32s/it]


In [ ]:
import os
import datetime
save_path = f'{PROJECT_ROOT}/results'
os.makedirs(save_path, exist_ok=True)

perf_metrics_df = pd.DataFrame(perf_metrics)
date_str = datetime.datetime.now().strftime('%Y-%m-%d_%H%M')
perf_metrics_df.to_csv(f'{save_path}/perf_metrics_full_{N_RUNS}_runs_{date_str}.csv')
print('Performance metrics saved')

Performance metrics saved
